# Average by genre

## Importation de modules

In [2]:
import numpy as np
import pandas as pd
from math import *
import matplotlib.pyplot as plt
import scipy

## Dataframe des métriques

In [3]:
metrics=['Accuracy','Precision micro', 'Recall micro', 'F1 score micro', 'Hamming-Loss']
final_res=pd.DataFrame(columns=metrics)

## Importation des données et mise en forme des dataframes

In [5]:
#importation des données
df0 = pd.read_parquet("/Users/sirinelouati/Desktop/dataset2.parquet", engine="pyarrow")
df = df0.copy()
df["Nombre labels"] = df.iloc[:,0:21].sum(axis=1)
df.sort_values("Nombre labels")

#dataframe des labels
labels = df.columns[:21]
df_labels = pd.DataFrame(data=labels,columns=['Label'])

In [6]:
#variables utiles
n_titre = len(df0) #nombre titres dans la base
n_audio = len(df0.loc[0,'audio_features']) #nombre de données audio
n_usage = len(df0.loc[0,'usage_features']) #nombre de données usage
n_label = len(df_labels) #nombre labels dans la base
p=0.5 #proportion de titres de la base utilisés

In [8]:
def create_labels(data, n_labels):
    labels = [[] for i in range(len(data))]
    labels_names = list(data.columns[:n_labels])
    for i in range(len(data)):
        vector = list(data.iloc[i][:n_labels])
        for j, elt in enumerate(vector):
            if elt:
                labels[i].append(labels_names[j])
    return labels

In [9]:
def keep_unique_label_data(df):
    df_unique = df
    to_drop = []
    labels = create_labels(df_unique, 21)
    for i in range(len(labels)):
        if len(labels[i]) > 1:
            to_drop.append(i)
    df_unique.drop(to_drop, 0, inplace=True)
    return df_unique

In [10]:
import pandas as pd

df0 = pd.read_parquet("dataset2.parquet", engine="pyarrow")
df = df0.copy()

#Ajout colonne nombre de labels pour chaque titre
df["Nombre labels"] = df.iloc[:,0:21].sum(axis=1)

# df_unilabel: la base avec les titres ne comportant qu'un seul label
df_unilabel = df[df["Nombre labels"] == 1]
df_unilabel.head(5)

,asian,rnb,reggae,blues,pop,dance,folk,arabic-music,indie,rock,...,jazz,electronic,african,country,song_index,audio_features,usage_features,artist_name,song_title,Nombre labels
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,195,"[0.7089933156967163, -0.6799708008766174, -0.3...","[-0.008137362965188247, -0.004262254068607103,...",Miles Davis,Générique,1
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,417,"[-0.19785162806510925, 0.023803016170859337, 0...","[0.00041993208544090015, 0.0001525178585704480...",Spin Doctors,Two Princes,1
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,530,"[-0.14142198860645294, 0.26422423124313354, 0....","[-0.009766703341977859, 0.0019064089803134893,...",Justin Bieber,One Less Lonely Girl,1
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,909,"[-0.07582522183656693, -0.42076417803764343, -...","[0.005023148381526522, 0.010775540737929749, 0...",Paul McCartney,My Love,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1257,"[-0.16135981678962708, -0.07056394964456558, -...","[0.00517276708433931, 0.006723451311827004, 0....",Jackson 5,Dancing Machine,1


In [11]:
def transform_parquet_to_csv(df,name_csv):
    #df=pd.read_parquet(file_path, engine="pyarrow")
    ind=df.index
    columns1=[]
    for i in range (1,257):
      col1= 'audio_feature_%i' % i
      columns1.append(col1)
    columns2=[]
    for i in range (1,129):
      col2= 'usage_feature_%i' % i
      columns2.append(col2)
    split_df_audio = pd.DataFrame(df['audio_features'].tolist(),columns=columns1,index=ind)
    split_df_usage=pd.DataFrame(df['usage_features'].tolist(),columns=columns2,index=ind)
    df1 = pd.concat([df, split_df_audio], axis=1)
    df =pd.concat([df1, split_df_usage], axis=1)
    df.drop(columns=['audio_features','usage_features'], inplace=True)
    df.set_index('song_index', inplace=True,drop=True)
    df.to_csv(name_csv)
    return df

In [12]:
transform_parquet_to_csv(keep_unique_label_data(df),'dataset.csv')

,asian,rnb,reggae,blues,pop,dance,folk,arabic-music,indie,rock,...,usage_feature_119,usage_feature_120,usage_feature_121,usage_feature_122,usage_feature_123,usage_feature_124,usage_feature_125,usage_feature_126,usage_feature_127,usage_feature_128
song_index,,,,,,,,,,,,,,,,,,,,,
195,0,0,0,0,0,0,0,0,0,0,...,0.008582,-0.000024,-0.272207,0.210503,0.030972,0.080964,-0.015423,0.102148,0.365081,-0.066897
417,0,0,0,0,0,0,0,0,0,1,...,-0.182309,-0.051225,-0.004617,-0.005726,-0.038334,-0.063972,0.053881,-0.001152,0.045032,0.027505
530,0,0,0,0,1,0,0,0,0,0,...,-0.044620,-0.132282,-0.141684,-0.106271,-0.189065,-0.229804,0.083824,-0.160790,0.040417,-0.094503
909,0,0,0,0,0,0,0,0,0,1,...,-0.127573,-0.068515,-0.106374,-0.102694,-0.066937,0.026911,-0.032158,0.063670,-0.026653,-0.068251
1257,0,0,0,0,0,0,0,0,0,0,...,0.024463,-0.052952,-0.059056,0.041670,-0.025673,-0.000446,-0.003438,0.060359,0.058938,0.029523
1312,0,0,0,0,0,0,0,0,0,0,...,0.033834,0.059702,0.127839,0.042218,-0.058068,-0.132771,0.048156,0.002076,0.139599,0.032241
1878,0,0,0,0,0,0,0,0,0,0,...,-0.017524,-0.030789,-0.035952,-0.053677,-0.015703,0.001386,0.011345,0.022860,0.039047,-0.013739
2090,0,0,0,0,0,0,0,0,0,0,...,-0.054643,0.010348,-0.306829,0.199233,0.056750,0.033910,-0.033535,0.071093,0.342383,-0.056093
2594,0,0,0,0,0,0,0,0,0,0,...,-0.041783,-0.068033,0.020318,0.011393,0.057474,-0.006398,0.040105,-0.014617,-0.119008,-0.025669


In [13]:
df=pd.read_csv('dataset.csv')

## Séparation train-test

In [15]:
from sklearn.model_selection import train_test_split

Y = df.iloc[:,1:22]
X = df[["audio_feature_"+str(i) for i in range(1,n_audio)]+["usage_feature_"+str(i) for i in range(1,n_usage)]]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state=17)

In [16]:
#le dataframe d'entraînement avec les sorties Y et les features X associées à chaque titre
df_train = pd.concat([X_train, y_train], axis = 1)

## Calcul des features moyennes pour chaque label

In [85]:
#le data frame avec les features moyennes de chaque label (moyenne calculée sur l'échantillon d'entraînement)
df_moyenne = pd.DataFrame(columns=df_train.columns)


In [53]:
for label in labels:
    df_moyenne.loc['Average_' + label] = df_train[df_train[label] == 1].mean(axis = 0)

In [54]:
df_moyenne = df_moyenne.iloc[:,0:n_audio+n_usage-2]
df_moyenne

,audio_feature_1,audio_feature_2,audio_feature_3,audio_feature_4,audio_feature_5,audio_feature_6,audio_feature_7,audio_feature_8,audio_feature_9,audio_feature_10,...,usage_feature_118,usage_feature_119,usage_feature_120,usage_feature_121,usage_feature_122,usage_feature_123,usage_feature_124,usage_feature_125,usage_feature_126,usage_feature_127
Average_asian,-0.001767,0.041335,0.307654,0.142246,-0.373179,-0.243955,-0.322277,0.970618,0.057668,0.628728,...,-0.020309,-0.052560,-0.011445,0.075815,0.024984,0.043598,0.045623,-0.000890,-0.021364,0.014642
Average_rnb,-0.071825,0.038172,0.324818,0.412570,-0.190021,-0.295830,-0.091879,1.175067,0.247276,0.532471,...,0.023674,-0.006198,-0.023662,-0.016335,0.008187,0.004008,0.002809,-0.016095,0.024209,0.021166
Average_reggae,-0.073774,-0.108953,0.191575,0.414199,-0.318465,-0.295402,-0.156414,0.940966,0.453997,0.417547,...,0.007395,0.001638,-0.006555,-0.011728,0.004932,0.012727,0.004249,0.004009,0.013548,0.022418
Average_blues,0.368261,-0.353981,-0.068903,0.218477,-0.420580,-0.274694,-0.370174,0.383734,0.298177,0.069259,...,0.020532,0.100077,0.160727,0.140443,0.030690,0.072946,-0.133935,-0.053404,0.060871,0.011486
Average_pop,-0.055695,0.105774,0.411500,0.338515,-0.376075,-0.282733,-0.188481,1.125204,0.165128,0.642738,...,0.022068,-0.026282,-0.026555,-0.044630,-0.046605,-0.011050,-0.052263,-0.006946,0.008572,0.040381
Average_dance,-0.157094,0.062862,0.338318,0.456279,-0.535044,-0.345313,-0.128015,1.208294,0.163241,0.584393,...,-0.010558,0.002904,0.001989,-0.020193,-0.008102,0.017167,0.021063,0.020292,0.009928,0.027262
Average_folk,0.262212,-0.214937,-0.133192,0.269594,-0.361031,-0.250223,-0.382602,0.771608,0.171485,0.404325,...,0.027635,0.065553,0.183307,-0.036754,-0.006486,-0.019732,0.141903,0.006602,-0.104071,0.163946
Average_arabic-music,-0.041389,-0.093111,0.319120,0.430863,-0.397376,-0.310481,-0.274200,0.938154,0.373936,0.416868,...,-0.000748,0.009748,0.010819,-0.005150,0.001300,0.031276,-0.008960,-0.009784,-0.003895,-0.008318
Average_indie,-0.094685,0.025534,0.094652,0.188710,-0.331755,-0.299539,-0.181833,1.060197,0.006981,0.603276,...,-0.006973,0.123385,0.040953,0.098395,-0.005265,0.026155,0.082614,0.006328,-0.023049,-0.019357
Average_rock,-0.071556,-0.013035,0.230501,-0.131437,-0.489255,-0.244735,-0.332772,0.678087,-0.112794,0.482426,...,0.021913,0.023064,0.041047,0.066856,-0.006784,0.037637,0.019078,0.001563,-0.038105,0.006024


In [25]:
#le dataframe de test avec les sorties Y et les features X associées à chaque titre
df_test = pd.concat([X_test, y_test], axis = 1)
df_test.head(5)

,audio_feature_1,audio_feature_2,audio_feature_3,audio_feature_4,audio_feature_5,audio_feature_6,audio_feature_7,audio_feature_8,audio_feature_9,audio_feature_10,...,latin,classical,k-pop,brazilian,metal,rap,jazz,electronic,african,country
36632,0.181751,-0.171783,-0.078838,0.320856,-0.668345,-0.297941,-0.288785,0.532548,0.467269,0.548710,...,0,0,0,1,0,0,0,0,0,0
12242,0.134305,0.220978,-0.069657,0.661069,-0.309832,-0.320165,-0.172844,0.801660,0.017179,0.951973,...,0,0,0,0,0,0,0,0,0,0
9322,-0.180728,0.101100,0.210772,0.732654,-0.043054,-0.320505,-0.106893,1.788445,0.021326,0.993075,...,0,0,0,0,0,0,0,0,0,0
2276,-0.179932,-0.016588,0.122550,0.392322,-0.509699,-0.339741,-0.127782,1.163137,0.028775,0.290782,...,0,0,0,0,0,1,0,0,0,0
30245,0.556450,0.142661,0.968847,0.583422,-0.441574,-0.076635,-0.507958,0.507932,0.939555,0.587549,...,0,0,0,0,0,0,0,0,0,0


In [58]:
df_test1 = df_test.iloc[:,0:n_audio+n_usage-2]
df_test1.shape[0]

12801

## Cosine similarity

In [59]:
import numpy as np
from numpy.linalg import norm
 

def cosine(a,b):
    A = np.array(a)
    B = np.array(b)
 
    return np.dot(A,B)/(norm(A)*norm(B))
    
    

## Labels prédits

In [79]:
y=[]
for i in range (df_test1.shape[0]):
    s=[]
    for j in range(21):
        a= cosine( df_moyenne.iloc[j,:],df_test1.iloc[i,:])
        s.append(a)
    for k in range(len(s)):
        if s[k]== min(s):
            y.append(k)
y

[15,
 12,
 12,
 17,
 15,
 15,
 12,
 17,
 12,
 19,
 15,
 15,
 12,
 17,
 12,
 12,
 15,
 12,
 15,
 12,
 17,
 10,
 12,
 12,
 12,
 15,
 17,
 12,
 17,
 15,
 15,
 17,
 16,
 15,
 12,
 12,
 12,
 12,
 10,
 17,
 12,
 12,
 16,
 17,
 16,
 12,
 15,
 15,
 12,
 17,
 12,
 16,
 15,
 17,
 12,
 12,
 19,
 12,
 15,
 18,
 12,
 17,
 12,
 17,
 12,
 18,
 17,
 15,
 15,
 17,
 15,
 17,
 12,
 12,
 17,
 17,
 16,
 19,
 15,
 12,
 3,
 17,
 15,
 17,
 12,
 15,
 15,
 15,
 17,
 15,
 17,
 12,
 12,
 17,
 15,
 13,
 12,
 13,
 18,
 15,
 17,
 12,
 19,
 12,
 12,
 15,
 17,
 15,
 12,
 12,
 15,
 15,
 12,
 12,
 16,
 15,
 13,
 15,
 12,
 15,
 16,
 12,
 15,
 13,
 17,
 12,
 17,
 12,
 12,
 13,
 13,
 12,
 17,
 15,
 12,
 13,
 12,
 17,
 15,
 12,
 12,
 12,
 17,
 17,
 17,
 12,
 12,
 15,
 12,
 13,
 20,
 13,
 12,
 15,
 12,
 15,
 15,
 15,
 12,
 15,
 12,
 16,
 12,
 13,
 12,
 12,
 12,
 15,
 17,
 12,
 12,
 12,
 17,
 15,
 17,
 12,
 17,
 17,
 12,
 2,
 12,
 17,
 15,
 17,
 12,
 19,
 12,
 15,
 12,
 12,
 12,
 15,
 12,
 17,
 12,
 15,
 12,
 12,
 17,
 16,
 1

In [86]:
labels[12]

'classical'

In [87]:
y_pred=[]
for i in range(len(y)):
    y_pred.append(labels[y[i]])

y_pred

['metal',
 'classical',
 'classical',
 'jazz',
 'metal',
 'metal',
 'classical',
 'jazz',
 'classical',
 'african',
 'metal',
 'metal',
 'classical',
 'jazz',
 'classical',
 'classical',
 'metal',
 'classical',
 'metal',
 'classical',
 'jazz',
 'soulfunk',
 'classical',
 'classical',
 'classical',
 'metal',
 'jazz',
 'classical',
 'jazz',
 'metal',
 'metal',
 'jazz',
 'rap',
 'metal',
 'classical',
 'classical',
 'classical',
 'classical',
 'soulfunk',
 'jazz',
 'classical',
 'classical',
 'rap',
 'jazz',
 'rap',
 'classical',
 'metal',
 'metal',
 'classical',
 'jazz',
 'classical',
 'rap',
 'metal',
 'jazz',
 'classical',
 'classical',
 'african',
 'classical',
 'metal',
 'electronic',
 'classical',
 'jazz',
 'classical',
 'jazz',
 'classical',
 'electronic',
 'jazz',
 'metal',
 'metal',
 'jazz',
 'metal',
 'jazz',
 'classical',
 'classical',
 'jazz',
 'jazz',
 'rap',
 'african',
 'metal',
 'classical',
 'blues',
 'jazz',
 'metal',
 'jazz',
 'classical',
 'metal',
 'metal',
 'metal',


In [88]:
len(y_pred)

12801